# CCS-AMP for unsourced multiple access

This notebook contains CCS-AMP encoder/decoder for unsourced multiple access using Hadamard design matrices.

The proposed algorithm goes back and forth between inner AMP and outer tree decoding components.

The code is based on the following articles:

A coded compressed sensing scheme for uncoordinated multiple access, available @ https://arxiv.org/pdf/1809.04745.pdf

SPARCs for Unsourced Random Access, available @ https://arxiv.org/abs/1901.06234

On Approximate Message Passing for Unsourced Access with Coded Compressed Sensing, available @ https://arxiv.org/pdf/2001.03705.pdf

Numbered equations are from https://arxiv.org/pdf/2010.04364.pdf

In [ ]:
import numpy as np
import ipdb
import matplotlib.pyplot as plt

In [ ]:
def fht(u):
    """
    Perform fast Hadamard transform of u, in-place.
    Note len(u) must be a power of two.
    """
    N = len(u)
    i = N>>1
    while i:
        for j in range(N):
            if (i&j) == 0:
                temp = u[j]
                u[j] += u[i|j]
                u[i|j] = temp - u[i|j]
        i>>= 1

def sub_fht(n, m, seed=0, ordering=None, new_embedding=False):
    """
    Returns functions to compute the sub-sampled Walsh-Hadamard transform,
    i.e., operating with a wide rectangular matrix of random +/-1 entries.

    n: number of rows
    m: number of columns

    It is most efficient (but not required) for max(m,n+1) to be a power of 2.

    seed: determines choice of random matrix
    ordering: optional n-long array of row indices in [1, max(m,n)] to
              implement subsampling; generated by seed if not specified,
              but may be given to speed up subsequent runs on the same matrix.

    Returns (Ax, Ay, ordering):
        Ax(x): computes A.x (of length n), with x having length m
        Ay(y): computes A'.y (of length m), with y having length n
        ordering: the ordering in use, which may have been generated from seed
    """
    assert n > 0, "n must be positive"
    assert m > 0, "m must be positive"
    if new_embedding:
        w = 2**int(np.ceil(np.log2(max(m+1, n+1))))
    else:
        w = 2**int(np.ceil(np.log2(max(m, n+1))))

    if ordering is not None:
        assert ordering.shape == (n,)
    else:
        rng = np.random.RandomState(seed)
        idxs = np.arange(1, w, dtype=np.uint32)
        rng.shuffle(idxs)
        ordering = idxs[:n]

    def Ax(x):
        assert x.size == m, "x must be m long"
        y = np.zeros(w)
        if new_embedding:
            y[w-m:] = x.reshape(m)
        else:
            y[:m] = x.reshape(m)
        fht(y)
        return y[ordering]

    def Ay(y):
        assert y.size == n, "input must be n long"
        x = np.zeros(w)
        x[ordering] = y.reshape(n)
        fht(x)
        if new_embedding:
            return x[w-m:]
        else:
            return x[:m]

    return Ax, Ay, ordering

def block_sub_fht(n, m, l, seed=0, ordering=None, new_embedding=False):
    """
    As `sub_fht`, but computes in `l` blocks of size `n` by `m`, potentially
    offering substantial speed improvements.

    n: number of rows
    m: number of columns per block
    l: number of blocks

    It is most efficient (though not required) when max(m,n+1) is a power of 2.

    seed: determines choice of random matrix
    ordering: optional (l, n) shaped array of row indices in [1, max(m, n)] to
              implement subsampling; generated by seed if not specified, but
              may be given to speed up subsequent runs on the same matrix.

    Returns (Ax, Ay, ordering):
        Ax(x): computes A.x (of length n), with x having length l*m
        Ay(y): computes A'.y (of length l*m), with y having length n
        ordering: the ordering in use, which may have been generated from seed
    """
    assert n > 0, "n must be positive"
    assert m > 0, "m must be positive"
    assert l > 0, "l must be positive"

    if ordering is not None:
        assert ordering.shape == (l, n)
    else:
        if new_embedding:
            w = 2**int(np.ceil(np.log2(max(m+1, n+1))))
        else:
            w = 2**int(np.ceil(np.log2(max(m, n+1))))
        rng = np.random.RandomState(seed)
        ordering = np.empty((l, n), dtype=np.uint32)
        idxs = np.arange(1, w, dtype=np.uint32)
        for ll in range(l):
            rng.shuffle(idxs)
            ordering[ll] = idxs[:n]

    def Ax(x):
        assert x.size == l*m
        out = np.zeros(n)
        for ll in range(l):
            ax, ay, _ = sub_fht(n, m, ordering=ordering[ll],
                                new_embedding=new_embedding)
            out += ax(x[ll*m:(ll+1)*m])
        return out

    def Ay(y):
        assert y.size == n
        out = np.empty(l*m)
        for ll in range(l):
            ax, ay, _ = sub_fht(n, m, ordering=ordering[ll],
                                new_embedding=new_embedding)
            out[ll*m:(ll+1)*m] = ay(y)
        return out

    return Ax, Ay, ordering

## Fast Hadamard Transforms

This code can all be found in `pyfht`, which uses a C extension to speed up the fht function. To make this notebook self contained, it's reproduced entirely in Python here, which will be quite slow!

Skip to the next section if you're not interested in the specific transform implementation.

In [ ]:
from pyfht import block_sub_fht

# Outer Tree encoder

This function encodes the payloads corresponding to users into codewords from the specified tree code. 

Parity bits in section $i$ are generated based on the information sections $i$ is connected to

Computations are done within the ring of integers modulo length of the section to enable FFT-based BP on the outer graph

This function outputs the sparse representation of encoded messages

In [ ]:
# tx_message is a (Ka x w) matrix containing Ka messages along its rows
# Ka is the number of active users
# messageBlocks is an indicator vector of which blocks are information blocks
# G is the generator matrix that defines the outer graph
# L is the number of sections 
# vl is the length of each coded section

def Tree_encode(tx_message, Ka, messageBlocks, G, L, vl):
    encoded_tx_message = np.zeros((Ka,L), dtype=int)            # instantiate (Ka x L) matrix to hold encoded message
    
    # Iterate through L blocks
    for i in range(0,L):
        
        # check to see if current block is an information block
        if messageBlocks[i]:
            
            # if so, directly take bits from tx_message for information block
            encoded_tx_message[:,i] = tx_message[:,np.sum(messageBlocks[:i])*vl:(np.sum(messageBlocks[:i])+1)*vl] \
                                      .dot(2**np.arange(vl)[::-1])
            
        # current block is a parity block.  Use equation (8)
        else:
            
            # nonzero elements of G[i] indicate which information blocks the current parity block is connected to
            indices = np.where(G[i])[0] 
            
            # instantiate data structure to hold parity information - (Ka x 1) vector of all zeros
            ParityInteger=np.zeros((Ka,1), dtype='int') 
            
            # sum of j in Wl (8)
            for j in indices:
                temp_summand = encoded_tx_message[:,j].reshape(-1,1)      # get column vector of messages in information block j
                ParityInteger = np.mod(ParityInteger+temp_summand,2**vl)  # perform addition over ring of integers mod 2^vl
                
            # store parity blocks inside encoded_tx_message
            encoded_tx_message[:,i] = ParityInteger.reshape(-1)
    
    # return encoded message
    return encoded_tx_message

This function converts message indices into $L$-sparse vectors of length $L 2^{v_{l}}$.

In [ ]:
# encoded_tx_message_indices is a (Ka x L) matrix holding the encoded messages for Ka active users
# L is the number of sections
# vl is the length of a coded section
# Ka is the number of active users

def convert_indices_to_sparse(encoded_tx_message_indices, L, vl, Ka):
    encoded_tx_message_sparse=np.zeros((L*2**vl,1), dtype=int)     # instantiate data structure for sparse coded messages
    
    # iterate over each block
    for i in range(L):
        A = encoded_tx_message_indices[:,i]                        # extract index representation of current block
        B = A.reshape([-1,1])                                      # convert from row vector to column vector
        np.add.at(encoded_tx_message_sparse, i*2**vl+B, 1)         # add sparse representation of block to encoded message
        
    # return sparse representation of encoded message
    return encoded_tx_message_sparse

This function returns the index representation corresponding to a SPARC-like vector.

In [ ]:
def convert_sparse_to_indices(cs_decoded_tx_message_sparse,L,J,listSize):
    cs_decoded_tx_message = np.zeros((listSize,L),dtype=int)
    for i in range(L):
        A = cs_decoded_tx_message_sparse[i*2**J:(i+1)*2**J]
        idx = (A.reshape(2**J,)).argsort()[np.arange(2**J-listSize)]
        B = np.setdiff1d(np.arange(2**J),idx)
        cs_decoded_tx_message[:,i] = B 

    return cs_decoded_tx_message

Extract information bits from retained paths in the tree.

In [ ]:
def extract_msg_indices(Paths,cs_decoded_tx_message, L,J):
    msg_bits = np.empty(shape=(0,0))
    L1 = Paths.shape[0]
    for i in range(L1):
        msg_bit=np.empty(shape=(0,0))
        path = Paths[i].reshape(1,-1)
        for j in range(path.shape[1]):
            msg_bit = np.hstack((msg_bit,cs_decoded_tx_message[path[0,j],j].reshape(1,-1))) if msg_bit.size else cs_decoded_tx_message[path[0,j],j]
            msg_bit=msg_bit.reshape(1,-1)
        msg_bits = np.vstack((msg_bits,msg_bit)) if msg_bits.size else msg_bit           
    return msg_bits


## SPARC Codebook

We use the `block_sub_fht` which computes the equivalent of $A.\beta$ by using $L$ separate $M\times M$ Hadamard matrices. However we want each entry to be divided by $\sqrt{n}$ to get the right variance, and we need to do a reshape on the output to get column vectors, so we'll wrap those operations here.

Returns two functions `Ab` and `Az` which compute $A\cdot B$ and $z^T\cdot A$ respectively.

In [ ]:
def sparc_codebook(L, M, n,P):
    Ax, Ay, _ = block_sub_fht(n, M, L, ordering=None)
    def Ab(b):
        return Ax(b).reshape(-1, 1)/ np.sqrt(n)
    def Az(z):
        return Ay(z).reshape(-1, 1)/ np.sqrt(n) 
    return Ab, Az

# BP on outer graph

This function computes the priors on the unknown sparse vector, given effective obervations of the (graph) neighboring sections

In [ ]:
# s = effective observation
# G = matrix that defines the graphical structure of outer code
# messageBlock = indicator vector of which blocks are information blocks
# l = number of sections
# ml = length of message section
# p0 = uninformative prior
# Ka = number of users
# τ = standard deviation of noise
# Phat = estimate of tx power
# numBPiter = number of belief propagation iterations to perform

# it appears that beta here refers to lambda in the paper...

# Notation has been adapted to match https://arxiv.org/pdf/2010.04364.pdf
def computePrior(s, G, messageBlocks, L, ml, p0, Ka, τ, Phat, numBPiter):
    
    # Initialize data structures
    q = np.zeros(s.shape,dtype=float)              # vector of priors to be returned to AMP algorithm
    p1 = p0*np.ones(s.shape,dtype=float)           # working estimate of priors to be used throughout BP algorithm
    μ = np.ones((L, 3, 2**vl))                     # data structure for storing messages passed on factor graph
    
    '''
    We will use a single data structure (μ) to hold all messages - both from information to parity nodes and parity to
    information nodes.  There will be one row for every section.  The row represents the node that sent the message.
    There are three columns - one for each of the nodes a node may be connected to.  At each row, column pair, a length 
    2**vl array is found representing the message sent from the node in the row position to the node in the column position. 
    
    Note that this collapsed matrix (i.e, L x 3 instead of L x L) will result in some interesting indexing operations.  The
    impetus behind using such a collapsed structure was the need to conserve memory. 
    '''

    # Compute local (instrinsic) estimate vector - see equation (38) and Lemma 3 (eq (30))
    temp_λ = (p1*np.exp(-(s-np.sqrt(Phat))**2/(2*τ**2))) / (p1*np.exp(-(s-np.sqrt(Phat))**2/(2*τ**2)) + (1-p1)*np.exp(-s**2/(2*τ**2))).astype(float).reshape(-1, 1)
    λ = temp_λ.reshape(L,-1)                       # (L, 2**vl) matrix
    λ = λ/(np.sum(λ,axis=1).reshape(L,-1))         # Normalize the rows of λ so that λ is a valid PME
    
    
    # Perform numBPiter iterations of the BP algorithm
    for iter in range(numBPiter):
        
        
        # Step 1: Compute messages μ(sl -> a) -- see equation (18)
        for r in range(L):
            if not messageBlocks[r]:               # skip parity blocks
                continue
            parityIdx = np.where(G[r])[0]          # determine which parity blocks the current info block is connected to 
            
            # compute messages from current info block to each of the above-identified parity blocks
            for c in range(len(parityIdx)):
                μ[r, c] = np.ones(2**vl)           # reset message to all "1"s
                μ[r, c] = μ[r, c] * λ[r,:]         # all messages get multiplied by λ
                
                # \prod_{a \in N(s_l) \ a} in eq (18)
                for d in range(len(parityIdx)):
                    if c != d:                     # ensure that we skip the recipient of the message
                        tmpIdx = np.where(G[parityIdx[d]])[0]
                        μ[r, c] = μ[r, c] * μ[parityIdx[d], np.where(tmpIdx == r)[0][0]]
                
                μ[r, c] = μ[r, c]/np.sum(μ[r, c])  # normalize answer
                
                
                
        # Step 2: Compute messages μ(ap -> s) -- see equation (17) and section "Design Considerations for Fast Execution"
        for r in range(L):
            if messageBlocks[r]:                    # skip info blocks
                continue
            infoIdx = np.where(G[r])[0]             # find all info blocks are connected to the current parity block
            μ[r, :] = np.ones(2**vl)                # reset message
            
            # compute messages from current parity block to each of the above-identified info blocks
            for j in range(len(infoIdx)):
                
                # sj \in N(ap) \ sl
                for k in range(len(infoIdx)):
                    if k != j:                      # setminus the recipient of the message
                        tmpIdx = np.where(G[infoIdx[k]])[0]
                        μ[r, k] = μ[r, k] * np.fft.fft(μ[infoIdx[k], np.where(tmpIdx == r)[0][0]])
                
                μ[r, j] = np.fft.ifft(μ[r, j]).real                        
                μ[r, j] = μ[r, j]/np.linalg.norm(μ[r, j], ord=0)  # normalize answer
                
        
        # Ensure that all values in μ are valid
        np.nan_to_num(μ)
    
    
    # After performing maxBPIter rounds of belief propagation, compute proper belief vector for AMP denoiser
    # see equation (39)
    for i in range(L):
        
        # get indices of blocks connected to current block
        blockIdx = np.where(G[i])[0]
        
        # initialize required data structure
        temp_p1 = np.ones(2**vl)
            
        # compute μ_{sl}
        for j in blockIdx:
            tempIdx = np.where(G[j])[0]
            temp_p1 = temp_p1 * μ[j, np.where(tempIdx == i)[0][0]]

        # use equation 39 to normalize properly
        p1[i*ml:(i+1)*ml] = (temp_p1/np.sum(temp_p1)).reshape(-1, 1)
        p1[i*ml:(i+1)*ml] = 1-(1-p1[i*ml:(i+1)*ml])**Ka
            
    
    # ensure that no value of p1 exceeds 1
    q = np.minimum(p1, 1)
    
    # return q
    return q


## AMP
This is the actual AMP algorithm. It's a mostly straightforward transcription from the relevant equations, but note we use `longdouble` types because the expentials are often too big to fit into a normal `double`.

In [ ]:
def amp(y, σ_n, P, L, ml, numAmpIter, Ab, Az, p0, Ka, G, messageBlocks, BPonOuterGraph, numBPiter):

    N = y.size                                  # dimension of vector y
    β = np.zeros((L*ml, 1))                     # initialize vector to store beta coefficients
    z = y.copy()                                # create deep copy of y for AMP algorithm
    Phat = N*P/L                                # Estimate of transmit power
    τ_evolution = np.zeros((numAmpIter,1))      # Store the values of τ corresponding to each iteration
    
    # perform T iterations of AMP
    for t in range(numAmpIter):
        
        τ = np.sqrt(np.sum(z**2)/N)                                         # Compute τ online using the residual
        s = (np.sqrt(Phat)*β + Az(z)).astype(np.longdouble)                 # effective observation
        
        # select proper prior
        if BPonOuterGraph==0:
            q = p0                                                          # Use the uninformative prior p0 for Giuseppe's scheme
        else:
            q = computePrior(s,G,messageBlocks,L,ml,p0,Ka,τ,Phat,numBPiter) # Compute the prior through BP on outer graph
            
        # denoiser
        β = (q*np.exp(-(s-np.sqrt(Phat))**2/(2*τ**2)))/ \
            (q*np.exp(-(s-np.sqrt(Phat))**2/(2*τ**2)) + \
            (1-q)*np.exp(-s**2/(2*τ**2))).astype(float).reshape(-1, 1)
        
        # residual
        z = y - np.sqrt(Phat)*Ab(β) + (z/(N*τ**2)) * (Phat*np.sum(β) - Phat*np.sum(β**2))
        
        # store tau value for debugging purposes
        τ_evolution[t] = τ

    # return β and τ_evolution
    return β, τ_evolution

# Outer Tree decoder

This function implements the tree deocoder for a specific graph corresponding to the outer tree code

It is currently hard-coded for a specfic architecture

The architecture is based on a tri-adic design and can be found in the simulation results section of https://arxiv.org/pdf/2001.03705.pdf

In [1]:
def Tree_decoder(cs_decoded_tx_message,G,L,J,B,listSize):
    
    tree_decoded_tx_message = np.empty(shape=(0,0))
    
    Paths012 = merge_paths(cs_decoded_tx_message[:,0:3])
    
    Paths345 = merge_paths(cs_decoded_tx_message[:,3:6])
    
    Paths678 = merge_paths(cs_decoded_tx_message[:,6:9])
    
    Paths91011 = merge_paths(cs_decoded_tx_message[:,9:12])
    
    Paths01267812 = merge_pathslevel2(Paths012,Paths678,cs_decoded_tx_message[:,[0,6,12]])
    
    Paths3459101113 = merge_pathslevel2(Paths345,Paths91011,cs_decoded_tx_message[:,[3,9,13]])
    
    Paths01267812345910111314 = merge_all_paths0(Paths01267812,Paths3459101113,cs_decoded_tx_message[:,[1,4,10,14]])
    
    Paths = merge_all_paths_final(Paths01267812345910111314,cs_decoded_tx_message[:,[7,10,15]])
    
    
   
    return Paths

def merge_paths(A):
    listSize = A.shape[0]
    B = np.array([np.mod(A[:,0] + a,2**16) for a in A[:,1]]).flatten()
     
    Paths=np.empty((0,0))
    
    for i in range(listSize):
        I = np.where(B==A[i,2])[0].reshape(-1,1)
        if I.size:
            I1 = np.hstack([np.mod(I,listSize).reshape(-1,1),np.floor(I/listSize).reshape(-1,1)]).astype(int)
            Paths = np.vstack((Paths,np.hstack([I1,np.repeat(i,I.shape[0]).reshape(-1,1)]))) if Paths.size else np.hstack([I1,np.repeat(i,I.shape[0]).reshape(-1,1)])
    
    return Paths

def merge_pathslevel2(Paths012,Paths678,A):
    listSize = A.shape[0]
    Paths0 = Paths012[:,0]
    Paths6 = Paths678[:,0]
    B = np.array([np.mod(A[Paths0,0] + a,2**16) for a in A[Paths6,1]]).flatten()
    
    Paths=np.empty((0,0))
    
    for i in range(listSize):
        I = np.where(B==A[i,2])[0].reshape(-1,1)
        if I.size:
            I1 = np.hstack([np.mod(I,Paths0.shape[0]).reshape(-1,1),np.floor(I/Paths0.shape[0]).reshape(-1,1)]).astype(int)
            PPaths = np.hstack((Paths012[I1[:,0]].reshape(-1,3),Paths678[I1[:,1]].reshape(-1,3),np.repeat(i,I1.shape[0]).reshape(-1,1)))
            Paths = np.vstack((Paths,PPaths)) if Paths.size else PPaths
               
    return Paths


def merge_all_paths0(Paths01267812,Paths3459101113,A):
    listSize = A.shape[0]
    Paths1 = Paths01267812[:,1]
    Paths4 = Paths3459101113[:,1]
    Paths10 = Paths3459101113[:,4]
    Aa = np.mod(A[Paths4,1]+A[Paths10,2],2**16)
    B = np.array([np.mod(A[Paths1,0] + a,2**16) for a in Aa]).flatten()
    
    Paths=np.empty((0,0))
    
    for i in range(listSize):
        I = np.where(B==A[i,3])[0].reshape(-1,1)
        if I.size:
            I1 = np.hstack([np.mod(I,Paths1.shape[0]).reshape(-1,1),np.floor(I/Paths1.shape[0]).reshape(-1,1)]).astype(int)
            PPaths = np.hstack((Paths01267812[I1[:,0]].reshape(-1,7),Paths3459101113[I1[:,1]].reshape(-1,7),np.repeat(i,I1.shape[0]).reshape(-1,1)))
            Paths = np.vstack((Paths,PPaths)) if Paths.size else PPaths
    
    return Paths

def merge_all_paths_final(Paths01267812345910111314,A):
    
    listSize = A.shape[0]
    Paths7 = Paths01267812345910111314[:,4]
    Paths10 = Paths01267812345910111314[:,11]
    B = np.mod(A[Paths7,0] + A[Paths10,1] ,2**16)
    
    Paths=np.empty((0,0))
    
    for i in range(listSize):
        I = np.where(B==A[i,2])[0].reshape(-1,1)
        if I.size:
            PPaths = np.hstack((Paths01267812345910111314[I].reshape(-1,15),np.repeat(i,I.shape[0]).reshape(-1,1)))
            Paths = np.vstack((Paths,PPaths)) if Paths.size else PPaths
    return Paths


If tree decoder outputs more than $K$ valid paths, retain $K-\delta$ of them based on their LLRs

$\delta$ is currently set to zero

In [2]:
def pick_topKminusdelta_paths(Paths, cs_decoded_tx_message, β, J,K,delta):
    
    L1 = Paths.shape[0]
    LogL = np.zeros((L1,1))
    for i in range(L1):
        msg_bit=np.empty(shape=(0,0))
        path = Paths[i].reshape(1,-1)
        for j in range(path.shape[1]):
            msg_bit = np.hstack((msg_bit,j*(2**J)+cs_decoded_tx_message[path[0,j],j].reshape(1,-1))) if msg_bit.size else j*(2**J)+cs_decoded_tx_message[path[0,j],j]
            msg_bit=msg_bit.reshape(1,-1)
        LogL[i] = np.sum(np.log(β[msg_bit])) 
    Indices =  LogL.reshape(1,-1).argsort()[0,-(K-delta):]
    Paths = Paths[Indices,:].reshape(((K-delta),-1))
    
    return Paths

# Simulation

In [ ]:
# Set up multiple access scenario
# Notation has been chosen to match https://arxiv.org/pdf/2010.04364.pdf
Ka = 25                 # Number of active users
w = 128                 # Payload size of each active user (per user message length)
L = 16                  # Number of sections/sub-blocks
N = 38400               # Total number of channel uses (real d.o.f)
vl = 16                 # Length of each coded sub-block
ml = 2**vl              # Length of each section of m

# Simulation Parameters
numAmpIter = 6          # Number of AMP iterations
listSize = 4 * Ka       # List size retained for each section after AMP converges
BPonOuterGraph = 1      # Indicates whether to perform BP on outer code.  If 0, AMP uses Giuseppe's uninformative prior
numBPiter = 2;          # Number of BP iterations on outer code
EbNodB = 2.4            # Energy per bit in decibels
maxSims=10              # Number of Simulations to Run

# Set Up Outer Tree Code
messageBlocks = np.array([1,1,0,1,1,0,1,1,0,1,1,0,0,0,0,0]).astype(int)    # Indicates the indices of information blocks
G = np.zeros((L,L)).astype(int)        # Define outer code factor graph via matrix G
G[0,[2,12]]=1
G[1,[2,14]]=1
G[2,[0,1]]=1
G[3,[5,13]]=1
G[4,[5,14]]=1
G[5,[3,4]]=1
G[6,[8,12]]=1
G[7,[8,15]]=1
G[8,[6,7]]=1
G[9,[11,13]]=1
G[10,[11,14,15]]=1
G[11,[9,10]]=1
G[12,[0,6]]=1
G[13,[3,9]]=1
G[14,[1,4,10]]=1
G[15,[7,10]]=1

print(G)

# Prepare Simulation
p0 = 1-(1-1/ml)**Ka     # Giuseppe's uninformative prior
EbNo = 10**(EbNodB/10)  # Eb/No in linear scale
P = 2*w*EbNo/N          # transmit power
σ_n = 1                 # Noise standard deviation
Phat = N*P/L            # Power estimate
msgDetected=0           # Track number of detected messages.  Used for error computation

In [ ]:
# Run CCS-AMP maxSims times
for sims in range(maxSims):
    
    # Seed RNG for consistency. Remove after testing.  The number 17 was chosen arbitrarily
    np.random.seed(17)
    
    # Generate messages for Ka active users
    tx_message = np.random.randint(low=2, size=(Ka, w))
    
    # Outer-encode the message sequences
    encoded_tx_message_indices = Tree_encode(tx_message, Ka, messageBlocks, G, L, vl)
    
    # Convert indices to sparse representation
    β_0 = convert_indices_to_sparse(encoded_tx_message_indices, L, vl, Ka)
    
    # Generate the binned SPARC codebook
    Ab, Az = sparc_codebook(L, ml, N, P)
    
    # Generate our transmitted signal X
    x = np.sqrt(Phat)*Ab(β_0)
    
    # Generate random channel noise and thus also received signal y
    z = np.random.randn(N, 1) * σ_n
    y = (x + z).reshape(-1, 1)
    
    # Run AMP decoding
    β, τ_evolution = amp(y, σ_n, P, L, ml, numAmpIter, Ab, Az,p0,Ka,G,messageBlocks,BPonOuterGraph,numBPiter)
    
    # FIXME: Remove the following line of code
    raise Exception('Stopping...')
    
    # Convert decoded sparse vector into vector of indices  
    cs_decoded_tx_message = convert_sparse_to_indices(β,L,vl,listSize)

    # Tree decoder to decode individual messages from lists output by AMP
    Paths = Tree_decoder(cs_decoded_tx_message,G,L,vl,w,listSize)
    
    # Re-align paths to the correct order
    perm = np.argsort(np.array([0,1,2,6,7,8,12,3,4,5,9,10,11,13,14,15]))
    Paths = Paths[:,perm]
    
    # If tree deocder outputs more than K valid paths, retain only K of them
    if Paths.shape[0] > Ka:
        Paths = pick_topKminusdelta_paths(Paths, cs_decoded_tx_message, β, vl, Ka, 0)

    # Extract the message indices from valid paths in the tree    
    Tree_decoded_indices = extract_msg_indices(Paths, cs_decoded_tx_message, L, vl)

    # Calculation of per-user prob err
    for i in range(Ka):
        msgDetected = msgDetected + np.equal(encoded_tx_message_indices[i,:],Tree_decoded_indices).all(axis=1).any()

    
errorRate= (Ka*maxSims - msgDetected)/(Ka*maxSims)

print("Per user probability of error = ", errorRate)